## AOC 2024/07
### Part 1

--- Day 7: Bridge Repair ---
The Historians take you to a familiar rope bridge over a river in the middle of a jungle. The Chief isn't on this side of the bridge, though; maybe he's on the other side?

When you go to cross the bridge, you notice a group of engineers trying to repair it. (Apparently, it breaks pretty frequently.) You won't be able to cross until it's fixed.

You ask how long it'll take; the engineers tell you that it only needs final calibrations, but some young elephants were playing nearby and stole all the operators from their calibration equations! They could finish the calibrations if only someone could determine which test values could possibly be produced by placing any combination of operators into their calibration equations (your puzzle input).

For example:

```
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
```

Each line represents a single equation. The test value appears before the colon on each line; it is your job to determine whether the remaining numbers can be combined with operators to produce the test value.

Operators are always evaluated left-to-right, not according to precedence rules. Furthermore, numbers in the equations cannot be rearranged. Glancing into the jungle, you can see elephants holding two different types of operators: add (+) and multiply (*).

Only three of the above equations can be made true by inserting operators:

- 190: 10 19 has only one position that accepts an operator: between 10 and 19. Choosing + would give 29, but choosing * would give the test value (10 * 19 = 190).
- 3267: 81 40 27 has two positions for operators. Of the four possible configurations of the operators, two cause the right side to match the test value: 81 + 40 * 27 and 81 * 40 + 27 both equal 3267 (when evaluated left-to-right)!
- 292: 11 6 16 20 can be solved in exactly one way: 11 + 6 * 16 + 20.
The engineers just need the total calibration result, which is the sum of the test values from just the equations that could possibly be true. In the above example, the sum of the test values for the three equations listed above is 3749.

Determine which equations could possibly be true. What is their total calibration result?

In [ ]:
sample_input="""
190: 10 19
3267: 81 40 27
83: 17 5
156: 15 6
7290: 6 8 6 15
161011: 16 10 13
192: 17 8 14
21037: 9 7 18 13
292: 11 6 16 20
""".strip()

**AI Prompt**

I'm a skilled python developer, please help go consider various approaches for solving this problem and their pros/cons. I prefer simpler and understandable solutions, easily readable also for other developers. It's ok to generate code with testcases without explanation. I'll ask if I do not understand it. Always wrap test code into a function, to not clutter the global variable namespace.

**AI Response**

For this problem, I see a few potential approaches:

1. **Brute Force with Product/Combinations**
   - Generate all possible operator combinations for each equation
   - Simple to understand and implement
   - Could use itertools.product to generate combinations
   - Might be slower for very long equations but perfect for this case

2. **Recursive Solution**
   - Solve by breaking down into subproblems
   - More elegant but might be harder to read
   - Overkill for this problem since equations are short

Let's start with a test implementation of the brute force approach since it's straightforward and maintainable. Would you like me to show you a basic test implementation?

**AI Prompt**

load full data

**AI Response**

Here's how we can load the actual puzzle input using the advent-of-code-data library:

```python
from aocd import get_data
data = get_data(day=7, year=2024)
print("First few lines of input:")
print("\n".join(data.splitlines()[:5]))
```

Would you like me to run this to verify the data format before we proceed with the solution?

In [ ]:
from aocd import get_data
full_input = get_data(day=7, year=2024)

**AI Prompt**

write function to parse input

**AI Response**

Here's a function to parse the input into a more manageable format:

```python
def parse_input(data):
    equations = {}
    for line in data.splitlines():
        if not line.strip(): continue
        result, nums = line.split(':')
        equations[int(result)] = [int(n) for n in nums.split()]
    return equations

def test_parse():
    parsed = parse_input(sample_input)
    assert parsed[190] == [10, 19]
    assert parsed[3267] == [81, 40, 27]
    assert len(parsed) == 9
    print("Parse test passed!")

test_parse()
```

This will create a dictionary where:
- Keys are the target values (before the colon)
- Values are lists of numbers to be combined

Would you like me to explain any part of this, or shall we move on to implementing the solution logic?

In [ ]:
def parse_input(data):
    equations = []
    for line in data.splitlines():
        if not line.strip(): continue
        result, nums = line.split(':')
        assert int(result) not in equations, f"Equation {result} duplicated"
        equations.append((int(result), [int(n) for n in nums.split()]))
    return equations

def test_parse():
    parsed = parse_input(sample_input)
    assert parsed[0] == (190, [10, 19])
    assert parsed[1] == (3267, [81, 40, 27])
    assert len(parsed) == 9
    print("Parse test passed!")

test_parse()

Parse test passed!


In [ ]:
sample_equations = parse_input(sample_input)
sample_equations

[(190, [10, 19]),
 (3267, [81, 40, 27]),
 (83, [17, 5]),
 (156, [15, 6]),
 (7290, [6, 8, 6, 15]),
 (161011, [16, 10, 13]),
 (192, [17, 8, 14]),
 (21037, [9, 7, 18, 13]),
 (292, [11, 6, 16, 20])]

In [ ]:
full_equations = parse_input(full_input)
len(full_equations)

850

In [ ]:
print(f"complexity: {sum([2**(len(eq[1])-1) for eq in full_equations])}")

complexity: 248472


**AI Prompt**

implement the brute force

**AI Response**

I'll help you implement the brute force solution using itertools.product for generating operator combinations. Here's the implementation:

```python
from itertools import product

def evaluate(nums, operators):
    result = nums[0]
    for i, op in enumerate(operators):
        if op == '+':
            result += nums[i + 1]
        else:  # op == '*'
            result *= nums[i + 1]
    return result

def can_make_value(target, nums):
    n_operators = len(nums) - 1
    for ops in product(['+', '*'], repeat=n_operators):
        if evaluate(nums, ops) == target:
            return True
    return False

def solve_part1(equations):
    return sum(target for target, nums in equations if can_make_value(target, nums))

def test_solution():
    assert solve_part1(sample_equations) == 3749
    print("Sample test passed!")

test_solution()
```

Would you like me to explain any part of this implementation, or shall we run it on the full input?

In [ ]:
from itertools import product

def evaluate(nums, operators):
    result = nums[0]
    for i, op in enumerate(operators):
        if op == '+':
            result += nums[i + 1]
        else:  # op == '*'
            result *= nums[i + 1]
    return result

def can_make_value(target, nums):
    n_operators = len(nums) - 1
    for ops in product(['+', '*'], repeat=n_operators):
        if evaluate(nums, ops) == target:
            return True
    return False

def solve_part1(equations):
    return sum(target for target, nums in equations if can_make_value(target, nums))

def test_solution():
    assert solve_part1(sample_equations) == 3749
    print("Sample test passed!")

test_solution()

Sample test passed!


In [ ]:
solve_part1(full_equations)

6392012777720

that was correct answer

### Part 2

--- Part Two ---
The engineers seem concerned; the total calibration result you gave them is nowhere close to being within safety tolerances. Just then, you spot your mistake: some well-hidden elephants are holding a third type of operator.

The concatenation operator (||) combines the digits from its left and right inputs into a single number. For example, 12 || 345 would become 12345. All operators are still evaluated left-to-right.

Now, apart from the three equations that could be made true using only addition and multiplication, the above example has three more equations that can be made true by inserting operators:

156: 15 6 can be made true through a single concatenation: 15 || 6 = 156.
7290: 6 8 6 15 can be made true using 6 * 8 || 6 * 15.
192: 17 8 14 can be made true using 17 || 8 + 14.
Adding up all six test values (the three that could be made before using only + and * plus the new three that can now be made by also using ||) produces the new total calibration result of 11387.

Using your new knowledge of elephant hiding spots, determine which equations could possibly be true. What is their total calibration result?

In [ ]:
print(f"complexity: {sum([3**(len(eq[1])-1) for eq in full_equations])}")

complexity: 14732280


In [ ]:
14732280 / 248472

59.29150970733121

**AI Prompt**

implement the updated problem

**AI Response**

Let's modify our solution to include the concatenation operator. Here's the updated implementation:

```python
def evaluate_part2(nums, operators):
    result = nums[0]
    for i, op in enumerate(operators):
        if op == '+':
            result += nums[i + 1]
        elif op == '*':
            result *= nums[i + 1]
        else:  # op == '||'
            result = int(str(result) + str(nums[i + 1]))
    return result

def can_make_value_part2(target, nums):
    n_operators = len(nums) - 1
    for ops in product(['+', '*', '||'], repeat=n_operators):
        if evaluate_part2(nums, ops) == target:
            return True
    return False

def solve_part2(equations):
    return sum(target for target, nums in equations if can_make_value_part2(target, nums))

def test_solution_part2():
    assert solve_part2(sample_equations) == 11387
    print("Sample test part 2 passed!")

test_solution_part2()
```

Would you like me to explain any part of this implementation before we run it on the full input?

In [ ]:
def evaluate_part2(nums, operators):
    result = nums[0]
    for i, op in enumerate(operators):
        if op == '+':
            result += nums[i + 1]
        elif op == '*':
            result *= nums[i + 1]
        else:  # op == '||'
            result = int(str(result) + str(nums[i + 1]))
    return result

def can_make_value_part2(target, nums):
    n_operators = len(nums) - 1
    for ops in product(['+', '*', '||'], repeat=n_operators):
        if evaluate_part2(nums, ops) == target:
            return True
    return False

def solve_part2(equations):
    return sum(target for target, nums in equations if can_make_value_part2(target, nums))

def test_solution_part2():
    assert solve_part2(sample_equations) == 11387
    print("Sample test part 2 passed!")

test_solution_part2()

Sample test part 2 passed!


In [ ]:
solve_part2(full_equations)

61561126043536